# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 9:55AM,264949,10,EYE43109,Eye Pharma Inc,Released
1,Jun 19 2023 9:55AM,264949,10,EYE43110,Eye Pharma Inc,Released
2,Jun 19 2023 9:55AM,264949,10,EYE43111,Eye Pharma Inc,Released
3,Jun 19 2023 9:54AM,264948,15,9586179,"Brookfield Pharmaceuticals, LLC",Released
4,Jun 19 2023 9:54AM,264948,15,9586180,"Brookfield Pharmaceuticals, LLC",Released
5,Jun 19 2023 9:54AM,264948,15,9586181,"Brookfield Pharmaceuticals, LLC",Released
6,Jun 19 2023 9:52AM,264947,15,9586169,Lonza,Released
7,Jun 19 2023 9:48AM,264945,10,9573616-BO-Hold_Rx(SC),Eywa Pharma Inc.,Released
8,Jun 19 2023 9:47AM,264944,10,9578461,Eywa Pharma Inc.,Released
9,Jun 19 2023 9:47AM,264944,10,9578986,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,264945,Released,1
20,264946,Released,50
21,264947,Released,1
22,264948,Released,3
23,264949,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264945,NaN,NaN,1.0
264946,NaN,NaN,50.0
264947,NaN,NaN,1.0
264948,NaN,NaN,3.0
264949,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264921,1.0,2.0,1.0
264925,0.0,1.0,0.0
264926,0.0,18.0,8.0
264927,0.0,1.0,0.0
264928,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264921,1,2,1
264925,0,1,0
264926,0,18,8
264927,0,1,0
264928,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264921,1,2,1
1,264925,0,1,0
2,264926,0,18,8
3,264927,0,1,0
4,264928,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264921,1,2,1
1,264925,,1,
2,264926,,18,8
3,264927,,1,
4,264928,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 9:55AM,264949,10,Eye Pharma Inc
3,Jun 19 2023 9:54AM,264948,15,"Brookfield Pharmaceuticals, LLC"
6,Jun 19 2023 9:52AM,264947,15,Lonza
7,Jun 19 2023 9:48AM,264945,10,Eywa Pharma Inc.
8,Jun 19 2023 9:47AM,264944,10,Eywa Pharma Inc.
10,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc."
60,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc."
65,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC"
71,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC"
72,Jun 19 2023 9:17AM,264940,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 19 2023 9:55AM,264949,10,Eye Pharma Inc,,,3
1,Jun 19 2023 9:54AM,264948,15,"Brookfield Pharmaceuticals, LLC",,,3
2,Jun 19 2023 9:52AM,264947,15,Lonza,,,1
3,Jun 19 2023 9:48AM,264945,10,Eywa Pharma Inc.,,,1
4,Jun 19 2023 9:47AM,264944,10,Eywa Pharma Inc.,,,2
5,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",,,50
6,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",,,5
7,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",,,6
8,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",,,1
9,Jun 19 2023 9:17AM,264940,10,Emerginnova,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:55AM,264949,10,Eye Pharma Inc,3,,
1,Jun 19 2023 9:54AM,264948,15,"Brookfield Pharmaceuticals, LLC",3,,
2,Jun 19 2023 9:52AM,264947,15,Lonza,1,,
3,Jun 19 2023 9:48AM,264945,10,Eywa Pharma Inc.,1,,
4,Jun 19 2023 9:47AM,264944,10,Eywa Pharma Inc.,2,,
5,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",50,,
6,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",5,,
7,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",6,,
8,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",1,,
9,Jun 19 2023 9:17AM,264940,10,Emerginnova,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:55AM,264949,10,Eye Pharma Inc,3,,
1,Jun 19 2023 9:54AM,264948,15,"Brookfield Pharmaceuticals, LLC",3,,
2,Jun 19 2023 9:52AM,264947,15,Lonza,1,,
3,Jun 19 2023 9:48AM,264945,10,Eywa Pharma Inc.,1,,
4,Jun 19 2023 9:47AM,264944,10,Eywa Pharma Inc.,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:55AM,264949,10,Eye Pharma Inc,3.0,NaN,NaN
1,Jun 19 2023 9:54AM,264948,15,"Brookfield Pharmaceuticals, LLC",3.0,NaN,NaN
2,Jun 19 2023 9:52AM,264947,15,Lonza,1.0,NaN,NaN
3,Jun 19 2023 9:48AM,264945,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Jun 19 2023 9:47AM,264944,10,Eywa Pharma Inc.,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:55AM,264949,10,Eye Pharma Inc,3.0,0.0,0.0
1,Jun 19 2023 9:54AM,264948,15,"Brookfield Pharmaceuticals, LLC",3.0,0.0,0.0
2,Jun 19 2023 9:52AM,264947,15,Lonza,1.0,0.0,0.0
3,Jun 19 2023 9:48AM,264945,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Jun 19 2023 9:47AM,264944,10,Eywa Pharma Inc.,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2384467,33.0,13.0,0.0
15,794841,54.0,0.0,0.0
19,1589584,16.0,20.0,0.0
20,529855,16.0,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2384467,33.0,13.0,0.0
1,15,794841,54.0,0.0,0.0
2,19,1589584,16.0,20.0,0.0
3,20,529855,16.0,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,13.0,0.0
1,15,54.0,0.0,0.0
2,19,16.0,20.0,0.0
3,20,16.0,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,15,Released,54.0
2,19,Released,16.0
3,20,Released,16.0
4,10,Executing,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,0.0,1.0
Executing,13.0,0.0,20.0,2.0
Released,33.0,54.0,16.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,0.0,1.0
1,Executing,13.0,0.0,20.0,2.0
2,Released,33.0,54.0,16.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,0.0,1.0
1,Executing,13.0,0.0,20.0,2.0
2,Released,33.0,54.0,16.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()